# Expected Completion

In [12]:
import pandas as pd
import numpy as np
from scipy.spatial.distance import cdist #function to create adjacency matrix
import math
import tensorflow
import spektral
from tensorflow.keras.layers import Dense
from tensorflow.keras.models import Model
from spektral.data import Dataset, BatchLoader, Graph, DisjointLoader
from spektral.transforms.normalize_adj import NormalizeAdj
from tensorflow.keras.layers import Dropout
from spektral.layers import GCNConv, GlobalSumPool

In [13]:
plays = pd.read_csv("data/pass_receiver.csv")
pff = pd.read_csv("data/pffScoutingData.csv")


read in weekly tracking files and concatenate into one tracking file, then delete weeklies

In [14]:
week1 = pd.read_csv("data/week1.csv")
week2 = pd.read_csv("data/week2.csv")
week3 = pd.read_csv("data/week3.csv")
week4 = pd.read_csv("data/week4.csv")
week5 = pd.read_csv("data/week5.csv")
week6 = pd.read_csv("data/week6.csv")
week7 = pd.read_csv("data/week7.csv")
week8 = pd.read_csv("data/week8.csv")
tracking = pd.concat([week1,week2,week3,week4,week5,week6,week7,week8])


In [15]:
#save memory
del week1,week2,week3,week4,week5,week6,week7,week8

In [16]:
tracking.columns

Index(['gameId', 'playId', 'nflId', 'frameId', 'time', 'jerseyNumber', 'team',
       'playDirection', 'x', 'y', 's', 'a', 'dis', 'o', 'dir', 'event'],
      dtype='object')

add unique identifier of each play and frame (some playIds repeat over games)

In [17]:
tracking["comb_id"] = tracking["gameId"].astype("str")+" "+tracking["playId"].astype("str")
pff["comb_id"] = pff["gameId"].astype("str")+" "+pff["playId"].astype("str")
plays["comb_id"] = plays["gameId"].astype("str")+" "+plays["playId"].astype("str")
tracking["comb_and_frame"] = tracking["gameId"].astype("str")+" "+tracking["playId"].astype("str")+" "+tracking["frameId"].astype("str")

isolate frames at time of pass

In [18]:
#isolate just forward passes and leave out a certain frame (duplicate forward pass for a play)
just_forward_passes = tracking[(tracking["event"]=="pass_forward") & (tracking["comb_and_frame"]!="2021091203 3740 32")& (tracking["team"]!="football")]
#bring in receiver info from plays
tracking_plays =just_forward_passes.merge(plays[["comb_id","possessionTeam","is_pass_failed","PassReceiver_nflId"]], how="left",on="comb_id" )
#and qb info from pff
tracking_merged_all = tracking_plays.merge(pff[["comb_id","nflId","pff_role"]], how = "left",on=["comb_id","nflId"])

In [19]:
len(just_forward_passes)

166034

In [20]:
len(tracking_plays)

166276

isolate tracking to just QB, OL, receiver and defense

In [21]:
tracking_players_filtered = tracking_merged_all[(tracking_merged_all["nflId"]==tracking_merged_all["PassReceiver_nflId"])|(tracking_merged_all["pff_role"]=="Pass")|(tracking_merged_all["pff_role"]=="Pass Block")|(tracking_merged_all["team"]!=tracking_merged_all["possessionTeam"])]

# Function to take in 1 pass play and put it in graph format( list containing adjacency matrix, features, y) this will then be parallelized for all passes in the data

In [22]:
#sample = tracking_players_filtered[tracking_players_filtered.comb_id == "2021090900 97"]
#data_dist = data[["x","y"]]

In [23]:
def angle_relative_diff(a,b):
    diff_list = []
    for i in range(len(a)):
        difference = a[i] - b[0]
        if difference < -180:
            difference +=360
        elif difference > 180:
            difference -=360
        diff_list.append(difference)
    return diff_list

In [24]:
def one_play_graph_function(data):
    ##one_play_formatted_list = []
    data = data.reset_index(drop=True)
    #get y, the response variable for this play (binary pass completed or not) and possesionteam (for CV)
    y = data["is_pass_failed"][0]
    team = data["possessionTeam"][0]
    #put play in adjacency matrix format, the distance of all players to each other (this will then be filtered to just link relevant players to QB or WR)
    data_dist = data[["x","y"]]
    adj = cdist(data_dist.values, data_dist.values)
    for index,row in data.iterrows():
        #connect QB to pass rush/block and wr, WR to coverage and QB, remove other connections by setting to zero
        if (row["pff_role"]=="Coverage" )| (row["pff_role"]=="Pass Route"):
            condition = ((data["pff_role"]=="Pass Block")|(data["pff_role"]=="Pass Rush"))
        else:
            condition = ((data["pff_role"]=="Coverage"))
        adj[index][condition] = 0
    #then put play in features format, getting a table containing position, speed, orientation etc
    data["onPossessionTeam"] = np.where(data["possessionTeam"]==data["team"],1,0)
    QB= data[data["pff_role"]=="Pass"]
    data["rel_o"] = data["o"]-QB["o"].iloc[0]
    data["rel_dir"] = data["dir"]-QB["dir"].iloc[0]
    data = data[["onPossessionTeam","s","rel_dir","rel_o","pff_role"]]
    one_hot = pd.get_dummies(data['pff_role'])
    data= data.drop('pff_role',axis = 1)
    # Join the encoded df
    data = data.join(one_hot)
    one_play_formatted_list = {"team":team,"y":y,"graph_a":adj,"graph_x":data}
    return one_play_formatted_list

In [25]:
import pickle
import multiprocessing
from joblib import Parallel, delayed

In [26]:
one_game = tracking_players_filtered[tracking_players_filtered.gameId == 2021090900]
one_game = one_game.reset_index()

In [28]:
graph_list = []
for i in range(len(tracking_players_filtered.comb_id.unique())):
#for i in range(10):
    
    one_play = tracking_players_filtered[tracking_players_filtered.comb_id==tracking_players_filtered.comb_id.unique()[i]]
    #print(one_game.comb_id.unique()[i])
    one_play = one_play.reset_index(drop=True)
    result = one_play_graph_function(one_play)
    graph_list.append(result.copy())


In [39]:
graph_list[7000]

{'team': 'IND',
 'y': 1,
 'graph_a': array([[ 0.        ,  0.        ,  0.        ,  0.        , 23.70248932,
          0.        , 17.17797718, 29.47224627,  0.        ,  0.        ,
          0.        ,  0.        ,  0.        , 27.01342259, 17.06435466,
          1.35915415, 23.42563553, 33.26386929],
        [ 0.        ,  0.        ,  6.26214021,  2.8900692 ,  2.85043856,
          2.95853004,  0.        ,  0.        ,  3.80131556,  0.86954011,
          5.67225705,  0.80024996,  3.12712328,  0.        ,  0.        ,
         25.96559454,  0.        ,  0.        ],
        [ 0.        ,  6.26214021,  0.        ,  3.80595586,  3.81903129,
          5.76042533,  0.        ,  0.        ,  6.21034621,  6.83714122,
          0.89403579,  7.05762   ,  3.88490669,  0.        ,  0.        ,
         21.68747334,  0.        ,  0.        ],
        [ 0.        ,  2.8900692 ,  3.80595586,  0.        ,  2.30670761,
          2.12268698,  0.        ,  0.        ,  2.82462387,  3.22024844,
   

get list of indexes of possession teams to allow masking for Cross Validation

In [37]:
team_list = []
for i in range(len(graph_list)):
    team_list.append(graph_list[i]["team"])

# GNN Functions

In [40]:
#variables
learning_rate = 1e-3  # Learning rate
epochs = 100  # Number of training epochs
es_patience = 10  # Patience for early stopping
batch_size = 21  # Batch size

{'team': 'TB',
 'y': 1,
 'graph_a': array([[ 0.        ,  3.73507697,  3.59777709,  3.78842975,  4.32786321,
          9.98721683,  0.        ,  0.        , 21.00506844,  0.        ,
         11.32525055,  0.        ,  6.27459959,  0.        ,  3.32085832,
          0.        ,  2.45079579,  1.78507703],
        [ 3.73507697,  0.        ,  4.0718055 ,  3.59680414,  3.61247837,
          7.16497732,  0.        ,  0.        , 17.3403835 ,  0.        ,
          8.41080852,  0.        ,  4.73305398,  0.        ,  2.83557754,
          0.        ,  2.84429253,  2.32845442],
        [ 3.59777709,  4.0718055 ,  0.        ,  0.69065187,  1.25251746,
          7.29414148,  0.        ,  0.        , 20.5315976 ,  0.        ,
          8.63229402,  0.        ,  3.09027507,  0.        ,  1.26051577,
          0.        ,  1.4707821 ,  4.17900706],
        [ 3.78842975,  3.59680414,  0.69065187,  0.        ,  0.62625873,
          6.69845505,  0.        ,  0.        , 19.8482367 ,  0.        ,
    

put data in graph format to feed in to GNN

In [ ]:
class GraphDataset(Dataset):
    def __init__(self, n_samples, df, n_outcome=1, **kwargs):
        self.n_samples = n_samples
        self.df = df  
        self.n_outcome = n_outcome
        super().__init__(**kwargs)

    def read(self):
        output = []
        for i in range(self.n_samples):
            # Node features
            iter_x = self.df[i]["graph_x"].copy()
            #print(iter_x)
            x = np.array(iter_x)#.reshape(iter_x.shape)

            # Edges
            iter_a =  self.df[i]["graph_a"].copy()
            the_length = len(iter_a)
            a = np.array(iter_a).reshape(the_length,the_length)
            #print(iter_a)
            y = int(self.df[i]["y"])
            
           
            output.append(Graph(x=x, a=a, y=y))
        return(output)

train xCompletion model

In [ ]:
class xCompletion(Model):

    def __init__(self, n_hidden, n_labels):
        super().__init__()
        self.graph_conv = GCNConv(n_hidden)
        self.pool = GlobalSumPool()
        self.dropout = Dropout(0.5)
        self.dense = Dense(n_labels, 'sigmoid')

    def call(self, inputs):
        out = self.graph_conv(inputs)
        out = self.dropout(out)
        out = self.pool(out)
        out = self.dense(out)

        return out

loop through and for each team use as val (remove from training) and store predictions

In [ ]:
prediction_list = []
for i in range(30):
    training_data = all but team[i]
    val_data = team[i]
    all_data = all
    len_train = len(training_data)
    len_val = len(val_data)
    len_all = len(local_all)
    data_tr = GraphDataset(n_samples = len_train, df = training_data, transforms=NormalizeAdj())
    data_va = GraphDataset(n_samples = len_val, df = val_data, transforms=NormalizeAdj())
    data_all = GraphDataset(n_samples = len_all, df=al_data, transforms=NormalizeAdj())

# Data loaders
    loader_tr = BatchLoader(data_tr, batch_size=batch_size, epochs=epochs)
    loader_va = BatchLoader(data_va, batch_size=batch_size)
    loader_all = BatchLoader(data_all, batch_size=batch_size)

    model = xCompletion(200, 1)
    model.compile('adam', "binary_crossentropy","binary_accuracy")

    model.fit(loader_tr.load(), validation_data= loader_va.load(), steps_per_epoch=loader_tr.steps_per_epoch,
    validation_steps=loader_va.steps_per_epoch, epochs=100)
    predictions = model.predict(loader_all.load(), steps =loader_all.steps_per_epoch)
    prediction_list.append(predicitions)
    